----------------------------------------
### 벡터검색 시나리오
이 시나리오는DB에 저장되어 있는 문서(pdf,hwpx)를 문장 임베딩 모델을 사용하여 유사 문서를 검색하는 데모입니다.

### 0. 파이썬 라이브러리 로딩 및 23ai 

In [ ]:
# 23ai 접속 및 라이브러리 로딩
import oracledb
import oml
import pandas as pd
import import_ipynb
from IPython.display import display
from user_functions import get_conn_cursor, get_db_version, exec_sql, dml_sql, ddl_sql, clean_text

# DB 접속
user_id = 'labadmin'
pwd = 'labadmin'
dbconn = 'dbserver26ai:1521/freepdb1'

conn, cursor = get_conn_cursor(user_id,pwd,dbconn)

# DB 버전 확인
get_db_version(cursor)

### 1. 샘플 테이블(doc_store) 데이터 조회
- docno varchar(10) primary key
- doc_dept varchar(40)
- doc_cr_date varchar(20)
- doc_name varchar2(200)
- doc blob

In [ ]:
# 샘플(doc_store) 테이블 데이터 조회

q1 = """ select docno, doc_dept, doc_cr_date, doc_name 
         from doc_store 
         order by 2, 1 """
exec_sql(cursor,q1)

### 2. 임베딩 모델 및 쿼리 임베딩 확인
- 이 데모에서는 DB에 저장된 엠베딩 모델을 사용합니다.
- 쿼리 임베딩 방법은 오라클 벡터 함수(dbms_vector_chain)을 사용하여 DB에 저장된 문서에서 텍스트 추출(utl_to_text)->청킹(utl_to_chunks)->임베딩(utl_to_embedding)을 단일 라인 코드로 실행

In [ ]:
# DB관리중인 ONNX 모델 조회
managed_models = """SELECT MODEL_NAME, MINING_FUNCTION, 
                    ALGORITHM, ALGORITHM_TYPE, (MODEL_SIZE/1024/1024) MB 
                    FROM user_mining_models 
                    ORDER BY MODEL_NAME"""

print("DB관리중인 ONNX 모델")
exec_sql(cursor,managed_models)

In [ ]:
# 임베딩 벡터 데이터 확인
print("임베딩 모델 : MULTILINGUAL_E5_SMALL")
get_vector = """
select dt.docno as docno,et.embed_id as chunk_id, et.embed_data as chunked_data, to_vector(et.embed_vector) embed_vector
from doc_store dt,
   dbms_vector_chain.utl_to_embeddings(
       dbms_vector_chain.utl_to_chunks(dbms_vector_chain.utl_to_text(dt.doc), 
       json('{ "by" : "words",
               "max" : "100", 
               "overlap" : "10",
               "split" : "newline", 
               "language" : "korean",
               "normalize": "all"}')),
       json('{"provider":"database", 
              "model":"MULTILINGUAL_E5_SMALL"}')) t,
   JSON_TABLE(t.column_value, 
             '$' columns (embed_id number,embed_data varchar2(4000), 
             embed_vector CLOB)) et
where dt.docno = :get_docno
fetch first 30 rows only
"""
pd.set_option('display.max_colwidth', 100)
result = exec_sql(cursor,get_vector,{"get_docno":"10-3"})
result = clean_text(result)
result['CHUNKED_DATA']=result['CHUNKED_DATA'].str.replace('\n', '', regex=False)
display(result)

### 3. 데이터 임베딩 및 인덱싱
- doc_store의 모든 문서의 텍스트를 추출, 워드 100개 단위로 청킹 후, 청크별로 벡터라이징(임베딩)하여 doc_store_chunks 테이블의 벡터 컬럼에 저장

In [ ]:
# 기존 데이터 삭제
remove_old_data = "truncate table doc_store_chunks"
dml_sql(conn,cursor,remove_old_data)

In [ ]:
# 임베딩 & 벡터 저장
load_vector = """
insert into doc_store_chunks
select dt.docno as docno,et.embed_id as chunk_id, et.embed_data as chunked_data, to_vector(et.embed_vector) embed_vector
from doc_store dt,
   dbms_vector_chain.utl_to_embeddings(
       dbms_vector_chain.utl_to_chunks(dbms_vector_chain.utl_to_text(dt.doc), 
       json('{ "by" : 
               "words",
               "max" : "100",
               "overlap" : "10",
               "split" : "newline", 
               "language" : "korean",
               "normalize": "all"}')),
       json('{"provider":"database", 
              "model":"MULTILINGUAL_E5_SMALL"}')) t,
   JSON_TABLE(t.column_value, 
              '$' columns (embed_id number,embed_data varchar2(4000), 
              embed_vector CLOB)) et
"""
print("벡터라이징 및 저장중입니다........")
dml_sql(conn,cursor,load_vector)

In [ ]:
# 벡터 인덱싱
ddl_sql(cursor,"drop index if exists doc_store_chunks_idx")

# 벡터 인덱싱
crIdx = """
create vector index doc_store_chunks_idx on doc_store_chunks(embed_vector)
ORGANIZATION INMEMORY NEIGHBOR GRAPH
WITH DISTANCE COSINE
"""
ddl_sql(cursor,crIdx)

# 벡터 인덱싱 조회
idx_q = """select table_name, index_name, index_type, index_subtype, status 
           from user_indexes where 
           table_name = 'DOC_STORE_CHUNKS'"""
exec_sql(cursor,idx_q)

### 4. 문서 유사성 검색

In [ ]:
# 검색 문장 입력 및 검색
pd.set_option('display.max_colwidth', 200)
print("예: 고향사랑 기부 상반기 모금 결과")
user_sentence = input("찾으실 문장을 입력하세요:")

sim_q = """
Select T.docno, D.doc_name, T.chunked_id, T.chunked_data AS 문서내용
from doc_store_chunks T, doc_store D
where D.docno = T.docno
order by vector_distance(T.embed_vector, 
                         dbms_vector.utl_to_embedding(:u_sentence, 
                               json('{"provider":"database", 
                                      "model":"MULTILINGUAL_E5_SMALL"}')),cosine)
fetch first 1 rows only
"""
sim_result = exec_sql(cursor,sim_q,{"u_sentence":user_sentence})
sim_result = clean_text(sim_result)
sim_result['문서내용']=sim_result['문서내용'].str.replace('\n', '', regex=False)
display(
    sim_result.head(10).style
    .set_properties(**{'text-align': 'left'})
        .set_table_styles([{'selector': 'th', 'props': [('text-align', 'left')]}])
)